In [ ]:
using Pkg
Pkg.activate("..")
using DrWatson, DataFrames, StatsPlots, Query, LinearAlgebra, Colors
include(srcdir("load.jl"))
ENV["GKS_WSTYPE"] = "png"
const outdir = projectdir("tmp", "plots", "distance")
mkpath(outdir)

In [2]:
results = collect_results(datadir("info", "te")) |>
    @filter(_.gh == 30 && _.gw == 30 && _.k == 3) |>
    DataFrame;

┌ Info: Scanning folder /home/ec2-user/dglmoore/yuji/data/info/te for result files.
└ @ DrWatson /home/ec2-user/.julia/packages/DrWatson/vLVOx/src/result_collection.jl:79
┌ Info: Added 972 entries.
└ @ DrWatson /home/ec2-user/.julia/packages/DrWatson/vLVOx/src/result_collection.jl:107


In [3]:
const W = 30;

In [4]:
distance(x, y) = norm(x .- y)

distance (generic function with 1 method)

In [5]:
δ = DataFrame(drug=String[], trial=Int[], phase=String[], distance=Float64[], te=Float64[])
for row in eachrow(results)
    te = linearize(row[:te])
    for i in 1:size(te, 1)
        for j in 1:size(te, 2)
            if te[j,i] >= te[i,j] && te[j,i] != zero(te[j,i])
                x = ((i - 1) ÷ W, (i - 1) % W)
                y = ((j - 1) ÷ W, (j - 1) % W)
                Δ = distance(x, y)
                push!(δ, [row[:drug], row[:trial], row[:phase], Δ, te[j,i] - te[i,j]])
            end
        end
    end
end

In [6]:
let kf = δ |> @filter(_.drug == "K-gluconate")
    knf = δ |> @filter(_.drug == "K-gluconate:Nifedipine")

    marker=(:circ, 2, 0.2)
    color = reshape(distinguishable_colors(2, colorant"green") |> reverse, 1, 2)

    kplot = @df kf dotplot(:distance, :te, group=:phase, marker=marker, color=color, markerstrokewidth=0)
    knplot = @df knf dotplot(:distance, :te, group=:phase, marker=marker, color=color, markerstrokewidth=0)

    plot(kplot, knplot, layout=2, title=["K-gluconate" "K-gluconate + Nifedipine"], titleloc=:left,
        xlabel="Distance", ylabel="Positive Net Transfer Entropy (bits)", size=(1000,500))
    savefig(joinpath(outdir, "tev.png"))
end